[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/analiza-dokumentow/blob/main/20--text-to-numeric.ipynb)

# Analiza dokumentów tekstowych	

# Reprezentacja numeryczna

https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text

https://neptune.ai/blog/vectorization-techniques-in-nlp-guide

https://www.matemaks.pl/wektory.html
https://pl.akinator.com/game

In [ ]:
from collections import namedtuple

Features = namedtuple("Feature", ["hair", "fat", "slim", "short", "tall", "like_honey", "live_in_forest"])

kubus = Features(0,1,0,1,0,1,1)
prosiaczek = Features(0,0,1,1,0,0,1)
osiol = Features(1,1,0,0,0,0,1)

answers = Features(1,1,0,0,1,1,1)

#[yes, probably_yes, idk, probably_no, no] = [-1, -0.5, 0, 0.5, 1]

In [ ]:
pip install scipy scikit-learn

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

## Bag of words
Pozwala reprezentować dane tekstowe jako wektor cech(ang. feature vector). Reprezentacja bag-of-words jest niezwykle prosta, sprowadza się do 2 kroków. 1. Stworzenie słownika unikalnych wyrazów - zbiór wyrazów z całej kolekcji dokumentów 2. Stworzenie reprezentacji wektorowej dla każdego z dokumentów zawierającej częstość wystąpień dla poszczególnych wyrazów

Jako, że pojedyńczy dokument zawiera wyłącznie mały wycinek z całego zbioru dostępnych wyrazów, wektor cech zawiera głównie 0. Często nazywany rzadkim (ang. sparse vector)

In [ ]:
documents = [
    "Care About Your Craft",
    "Make Quality a Requirements Issue",
    "Don't Repeat Yourself",
    "Always Design for Concurrency",
    "Sign Your Work",
    "Refactor Early, Refactor Often",
]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [ ]:
cv = CountVectorizer(binary=True)
cv.fit(documents)
X = cv.transform(documents)

In [ ]:
cv.get_feature_names_out()

In [ ]:
DF = pd.DataFrame(X.toarray(), columns=cv.get_feature_names_out())
display(DF)

Transformacja pozwoliła uzyskać strukturę w której każdy wiersz reprezentuje dokument, każda kolumna określa częstość wystąpienia danego tokenu

## Wektor częstości

In [ ]:
cv = CountVectorizer(binary=False)
cv.fit(documents)
X = cv.transform(documents)
DF = pd.DataFrame(X.toarray(), columns=cv.get_feature_names_out())
display(DF)

### zalety
+ Jest prostą numeryczną reprezentacją danych tekstowych.

### wady

- Problem stanowi rozmiar wektora który jest równy długości całego słownika.
- Nie uwzględnia relacji pomiędzy semantyką wyrazów
- relacje pomiędzy słowami - rózne interpretacje
- znaczenie – te same słowa różne znaczenie
- metafory, tautologie, ironie
- znaki specialne
- homonimy - to samo brzmienie inne znaczenia
- synonimy, idiomy
- 150k słów w słowniku


## Ważony wektor częstości
 

TF-IDF Term Frequency - Inverse Document Frequency pozwala uzyskać wartość określającą istotność danego tokenu w kontekście całego dokumentu. 
Wartość częstości (TF) nie uwzględnia różnicy w wartości informacyjnej jaką niosą ze sobą poszczególne wyrazy.
IDF - pozwala tą różnicę uchwycić.

Innymi słowy metodad TF-IDF pozwala wzmocnić znaczenie tokenu wraz ze wzrostem ilości wystąpień w dokumencie. Jednocześnie niwelując efekt wzmocnienia jeżeli dany token występuje równie często w pozostałych dokumentach stanowiących korpus.

Wartość tf-idf może być policzona przez mnożenie wartości tf oraz idf

$$ TfIdf(t,d) = tf(t,d)*idf(t,d)$$


$tf(t,d)$ jest częstością występowania tokenu w dokumencie
$idf(t,d)$ obliczany jest wg wzoru

$$ idf(t,d) = log\frac{n_d}{1+df(d,t)}$$
$n_d$ - liczba wystąpień w korpusie

$df(d,t)$ - liczba dokumentów gdzie występuje token t

$1$ - stała 1 jest opcjonalna, niemniej pozwala uzyskać niezerowe wartości dla tokenów występujących we wszystkich dokumentach


Implementacja TF-IDF w scikit-learn różni się jednak od powyższej definicji jest liczony następująco:
$$ TfIdf(t,d) = tf(t,d)*(idf(t,d) + 1)$$
$$ idf(t,d) = log\frac{1 + n_d}{1+df(d,t)}$$


In [ ]:
documents = [
    "Hey you, Test Early. Test Often. Test Automatically!",
    "Hey you, Refactor Early, Refactor Often, Just as you might!",
    "Hey you, Abstractions Live Longer than Details",
    "Hey you, Use Saboteurs to Test Your Testing",
    "Hey you, keep Knowledge in Plain Text",
]

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

tdif = TfidfVectorizer()
X = tdif.fit(documents)
X = tdif.transform(documents)
DF = pd.DataFrame(X.toarray(), columns=tdif.get_feature_names_out())
display(DF)

### zalety

* wielkość dokumentu nie ma wpływu wartość, tfifd która jest normalizowana
* uwzględnia różnice w znaczeniu tokenów dla znaczenia dokumentu

### wady

- Nie uwzględnia relacji pomiędzy semantyką wyrazów
- relacje pomiędzy słowami - rózne interpretacje
- znaczenie – te same słowa różne znaczenie
- homonimy - to samo brzmienie inne znaczenia
- ignoruje kolejność wyrazów w dokumencie

## Wektory - odległość

https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cosine.html

### Podobieństwo wektorów / odległość kosinusowa

Jest metryką pozwalającą zmierzyć jak 2 wektory są podobne względem siebie. Wartość otrzymujemy obliczając wartość kosinusa konta pomiędzy wektorami w przestrzeni. Odległosć kosinusowa pozwala zniwelować wartości wynikające z położenia wektorów w przestrzeni, zachowując interpretowalną statystykę podobieństwa. Zakres wartości (0,1)



![cosine](http://blog.jkan.pl/images/cosine-distance.svg)

$ A = (3,3)$
$ B = (5,2)$
$ C = (5,0.5)$

$$Cos\theta = \frac{\vec{a} * \vec{b}}{||\vec{a}|| ||\vec{b}||} = \frac{\sum_1^n a_ib_i}{\sqrt{\sum_1^na_i^2} * \sqrt{\sum_1^nb_i^2}}$$
$$Cos\theta = \frac{15 + 6}{\sqrt{18} * \sqrt{29}}$$
$$Cos\theta = 0,9191$$

$$cos(90) = 0$$
$$cos(0) = 1$$
Kosunius przyjmuje wartość ``0`` dla wektorów prostopadłych (ortogonalnych) i wartość ``1`` dla wektorów równoległych. Im wartość kosinusa bliższa ``1`` tym mniejszy kont pomiędzy wektorami

Wykorzystując metodę cosine z ``scipy.spatial.distance`` musimy pamiętać o przekształceniu wg wzoru: 

$$Cosine Similarity = 1 − Cosine Distance$$

Oczekujemy że $podobieństwo(A,B) > podobieństwo(A,C)$ ponieważ kąt pomiędzy wektorami A,B jest mniejszy



In [ ]:
from scipy.spatial.distance import cosine
A = (3,3)
B = (5,2)
C = (5,0.5)
cosine(A, B), cosine(A, C), cosine(A ,A), cosine((0,1), (1,0))

In [ ]:
## similarity 1 - cosine
1 - cosine(kubus,kubus) # maxymalne podobieństwo

In [ ]:
A = (1,0,0)
B = (0,1,0)
1 - cosine(A,B)
0.0

In [ ]:
A = (1,0,0)
1 - cosine(A,A)
1.0

In [ ]:
characters = [('kubus', kubus), ('prosiaczek', prosiaczek), ('osiol', osiol)]

akinator = list(map(lambda vec: (vec[0], 1 - cosine(vec[1], answers)), characters))

sorted(akinator, key=lambda vec: vec[1], reverse=True)

In [ ]:
def play_the_game(answers: Features) -> str:
    characters = [('kubus', kubus), ('prosiaczek', prosiaczek), ('osiol', osiol)]
    akinator = list(map(lambda vec: (vec[0], 1 - cosine(vec[1], answers)), characters))
    guess = sorted(akinator, key=lambda vec: vec[1], reverse=True)
    return guess[0]

In [ ]:
play_the_game(Features(0,0,1,1,0,0,0))

## Inne metryki
* Manhattan distance
* Euclidean distance
* Minkowski distance
* Jaccard similarity

In [ ]:
from scipy.spatial.distance import jaccard
from scipy.spatial.distance import minkowski
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import cityblock
from scipy.spatial.distance import cosine

A = (3, 22, 55, 13)
B = (1, 12, 40, 5)

jaccard(A,B), minkowski(A,B), euclidean(A,B), cityblock(A,B), cosine(A,B)
(1.0, 19.82422760159901, 19.82422760159901, 35, 0.008847457489864041)